# Load dataset into Snowflake

***This notebook works well with the `conda_python3` kernel on a SageMaker Notebook `ml.t3.xlarge` instance.***

---

In this notebook we will do the following:
1. Install dependencies i.e. Python packages needed to connect to Snowflake.
1. Connect to Snowflake and create a dateware and a database.
1. Download the [California housing dataset](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html) from scikit-learn and ingest it into a new Snowflake table.


---

## Install dependencies 

In [ ]:
!pip install snowflake-connector-python==2.9.0
!pip install pyarrow==10.0.1

---

## Connect to Snowflake account


Now you need to retrieve your account identifier for Snowflake. This can be done by looking into Snowflake account and then copying the account URL from the bottom right corner of the page as shown in the screenshot below.

![](./img/snowflake-accountid-1.png)

The account URL is of the form `https://abc12345.us-east-1.snowflakecomputing.com` the account identifier is the `abc12345.us-east-1` part. _Note that in your case the `us-east-1` might be replaced with a different region_.

In [ ]:
sf_account_id = "your-snowflake-account-id"

### Retrieve Snowflake credentials from AWS secrets manager

The following code retrieves your Snowflake username and password from a secret stored called `snowflake_credentials` store in AWS Secrets Manager. The secret is expected to have two key-value pairs: `username`: this is your Snowflake username and `password`: this is the password associated with your Snowflake username. For instructions on how to create a secret in AWS Secrets Mananger refer to [`Create an AWS Secrets Mananger secret`](https://docs.aws.amazon.com/secretsmanager/latest/userguide/create_secret.html).

In [ ]:

import os
import json
import boto3
import pandas as pd 
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# collect snowflake credentials from Secrets Manager
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='snowflake_credentials')
secrets_credentials = json.loads(response['SecretString'])
sf_password = secrets_credentials['password']
sf_username = secrets_credentials['username']

In [ ]:
sf_username

### Connect

In [ ]:
# connect to Snowflake account
conn = snowflake.connector.connect(
    user        = sf_username,
    password    = sf_password,
    account     = sf_account_id,
    protocol    = "https")

## Create Snowflake data warehouse and database

In [ ]:
dwh = "amazon_sagemake_w_snowflake_as_datasource"
db = "housing"
schema = "housing_schema"
table = "california_housing"

In [ ]:
conn.cursor().execute(f"CREATE WAREHOUSE IF NOT EXISTS {dwh}")
conn.cursor().execute(f"USE WAREHOUSE {dwh}")
conn.cursor().execute(f"CREATE DATABASE IF NOT EXISTS {db}")
conn.cursor().execute(f"USE DATABASE {db}")

---

## Download the dataset from scikit-learn

In [ ]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)
california_housing = california_housing.frame
california_housing.head()

---

## Ingest data into the Snowflake table

In [ ]:
# connect to Snowflake Table schema
conn.cursor().execute(f"CREATE SCHEMA IF NOT EXISTS {schema}")
conn.cursor().execute(f"USE SCHEMA {schema}")


create_table_sql = f"CREATE TABLE IF NOT EXISTS {db}.{schema}.{table}\n ("

california_housing.rename(columns=str.upper, inplace=True)
california_housing.columns

# iterating through the columns
for col in california_housing.columns:
    column_name = col.upper()

    if (california_housing[col].dtype.name == "int" or california_housing[col].dtype.name == "int64"):
        create_table_sql = create_table_sql + column_name + " int"
    elif california_housing[col].dtype.name == "object":
        create_table_sql = create_table_sql + column_name + " varchar(16777216)"
    elif california_housing[col].dtype.name == "datetime64[ns]":
        create_table_sql = create_table_sql + column_name + " datetime"
    elif california_housing[col].dtype.name == "float64":
        create_table_sql = create_table_sql + column_name + " float8"
    elif california_housing[col].dtype.name == "bool":
        create_table_sql = create_table_sql + column_name + " boolean"
    else:
        create_table_sql = create_table_sql + column_name + " varchar(16777216)"

    # deciding next steps. Either column is not the last column (add comma) else end create_tbl_statement
    if california_housing[col].name != california_housing.columns[-1]:
        create_table_sql = create_table_sql + ",\n"
    else:
        create_table_sql = create_table_sql + ")"

        # execute the SQL statement to create the table
        print(f"create_table_sql={create_table_sql}")
        conn.cursor().execute(create_table_sql)  
print(f"snowflake_table={table}")
conn.cursor().execute(f'TRUNCATE TABLE IF EXISTS {table}')    




In [ ]:
print(f"database={db}, schema={schema}, snowflake_table={table}")
# Write the data from the DataFrame to the Snowflake table.
write_pandas(conn=conn,
             df=california_housing,
             table_name=table.upper(),
             database=db.upper(),
             schema=schema.upper())